In [1]:
import torch
import torchvision.models as models
import torch.nn.functional as F
from torch import nn
import math
import os
import numpy as np

In [2]:
import yaml
import argparse
import datetime

from utilities.model_helper import build_model
from utilities.dataloader_helper import build_dataloader
from utilities.optimizer_helper import build_optimizer
from utilities.scheduler_helper import build_lr_scheduler
from utilities.trainer_helper import Trainer
from utilities.tester_helper import Tester
from utilities.utils_helper import create_logger
from utilities.utils_helper import set_random_seed


c:\Users\strom\Desktop\monodle\utilities\datasets\kitti\kitti_eval_python\eval.py:10: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_thresholds(scores: np.ndarray, num_gt, num_sample_pts=41):


>>CONFIG

In [3]:
cfg = {
    'random_seed': 444,
    'dataset': {
        'type': 'KITTI',
        'batch_size': 16,
        'use_3d_center': True,
        'class_merging': False,
        'use_dontcare': False,
        'bbox2d_type': 'anno',
        'meanshape': False,
        'writelist': ['Car'],
        'random_flip': 0.5,
        'random_crop': 0.5,
        'scale': 0.4,
        'shift': 0.1
    },
    'model': {
        'type': 'centernet3d',
        'backbone': 'dla34',
        'neck': 'DLAUp',
        'num_class': 3
    },
    'optimizer': {
        'type': 'adam',
        'lr': 0.00125,
        'weight_decay': 1e-05
    },
    'lr_scheduler': {
        'warmup': True,
        'decay_rate': 0.1,
        'decay_list': [90, 120]
    },
    'trainer': {
        'max_epoch': 140,
        'gpu_ids': '0',
        'save_frequency': 10
    },
    'tester': {
        'type': 'KITTI',
        'mode': 'single',
        'checkpoint': '/Users/strom/Desktop/monodle/utilities/checkpoints/checkpoint_epoch_140.pth',
        'checkpoints_dir': 'checkpoints',
        'threshold': 0.2
    },
    'evaluate': True,
}

>> TOTAL CENTER NET MODEL

In [4]:
train_loader, test_loader  = build_dataloader(cfg['dataset'])

In [5]:
log_file = 'train.log.%s' % datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
logger = create_logger(log_file)

In [6]:
model = build_model(cfg['model'])
print(model)

CenterNet3D(
  (backbone): DLA(
    (base_layer): Sequential(
      (0): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (level0): Sequential(
      (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (level1): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (level2): Tree(
      (tree1): BasicBlock(
        (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [14]:
if cfg['evaluate']:
        tester = Tester(cfg=cfg['tester'],
                        model=model,
                        dataloader=test_loader,
                        logger=logger)
        tester.test()

2023-05-15 17:59:28,322   INFO  ==> Done
Evaluation Progress: 100%|██████████| 236/236 [02:05<00:00,  1.88it/s]
2023-05-15 18:01:33,879   INFO  ==> Saving ...
2023-05-15 18:01:36,440   INFO  ==> Loading detections and GTs...
2023-05-15 18:01:37,490   INFO  ==> Evaluating (official) ...
c:\Users\strom\anaconda3\envs\PyTorch\lib\site-packages\numba\cuda\dispatcher.py:539: NumbaPerformanceWarning: Grid size 36 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
c:\Users\strom\anaconda3\envs\PyTorch\lib\site-packages\numba\cuda\dispatcher.py:539: NumbaPerformanceWarning: Grid size 50 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
c:\Users\strom\anaconda3\envs\PyTorch\lib\site-packages\numba\cuda\dispatcher.py:539: NumbaPerformanceWarning: Grid size 32 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
c:\Users\strom\anaconda3\envs\Py

In [7]:
#  build optimizer
optimizer = build_optimizer(cfg['optimizer'], model)

# build lr scheduler
lr_scheduler, warmup_lr_scheduler = build_lr_scheduler(cfg['lr_scheduler'], optimizer, last_epoch=-1)

In [8]:
logger.info('###################  Training  ##################')
logger.info('Batch Size: %d'  % (cfg['dataset']['batch_size']))
logger.info('Learning Rate: %f'  % (cfg['optimizer']['lr']))
trainer = Trainer(cfg=cfg['trainer'],
                    model=model,
                    optimizer=optimizer,
                    train_loader=train_loader,
                    test_loader=test_loader,
                    lr_scheduler=lr_scheduler,
                    warmup_lr_scheduler=warmup_lr_scheduler,
                    logger=logger)
trainer.train()

2023-05-15 18:09:18,174   INFO  ###################  Training  ##################
2023-05-15 18:09:18,176   INFO  Batch Size: 16
2023-05-15 18:09:18,177   INFO  Learning Rate: 0.001250
epochs:   0%|          | 0/140 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 60.00 MiB (GPU 0; 6.00 GiB total capacity; 5.27 GiB already allocated; 0 bytes free; 5.29 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

>>check device

>>LOAD WEIGHTS

>>IMPORT FOR DATALOADER

>>DATALOADER

>>INFERENCE